In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from transformers import AutoTokenizer
from pyvi import ViTokenizer
from itertools import chain
from rank_bm25 import BM25Okapi
from difflib import SequenceMatcher
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
from sklearn.cluster import KMeans
import regex as re
import matplotlib.pyplot as plt
from collections import Counter
import unicodedata
import scipy
import sklearn
import math
import os
import json
from itertools import chain
#from rich import print
from collections import defaultdict


In [2]:
folder_path = r'C:\\Users\\nguye\\Downloads\\new_processed_hotels\\new_processed_hotels'

data_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            data_list.append(data)

        except json.JSONDecodeError as e:
            print(f'Không thể phân tích cú pháp tệp {filename}: {e}')
        except Exception as e:
            print(f'Đã xảy ra lỗi với tệp {filename}: {e}')

In [5]:
data_list[0]['info']['rate']

'9,2'

In [7]:
results = []
for data in data_list:
    results.append({"id": data['id'], 
                    "stars_rating": data['stars_rating'],
                    "sub_rate": data['info']['sub_rate'],
                    "avg_rating": data['info']['rate'],
    })

In [9]:
with open('feature_star_rating.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [10]:
import get_score_price_star

In [26]:
ids_result = []
for id in results[:1000]:
    ids_result.append(id['id'])

In [13]:
user_input = {
    # Input địa điểm
    'province': 'TP Hồ Chí Minh',
    'district': None,
    'nearby_places': ["Chợ Bến Thành", "Dinh Độc Lập", "Nhà thờ Đức Bà"], # Bao gồm địa điểm tham quan + địa điểm di chuyển
    'is_near_center': True,
    # Input thông tin tổng quát
    'stars_rating': 3,
    'price': (800000, 2000000),
    'rating': 8,
    'sub_rate': {
        "staff": 9.0,
        "facilities": 8.3,
        "cleanliness": 8.7,
        "comfort": 8.7,
        "value": 8.4,
        "location": 8.4,
        "free WiFi": 8.3
    },
    # Input thông tin về cơ sở vật chất và tiện nghi
    'services': [],  # 1 trong 18 loại dịch vụ được giới thiệu trước
    'amenities': ["TV màn hình phẳng, két an toàn"],
    # Input thông tin về phòng
    'capacity': 2,
    'room_type': 'Family',
    'area': 50,
    'bed_type': '',
    'included_breakfast': True,
    'room_amenties':[],
    'room_facilities':[],
    'room_view': [],

    # Input thông tin về chính sách
    'policies': {
        "Nhận phòng": '11:00-12:00',
        "Trả phòng": '12:00-13:00',
        "Chỉ thanh toán bằng tiền mặt": None,
        "Các phương thức thanh toán được chấp nhận": 'Bankcard',
        "Giới hạn độ tuổi": None,
        "Giờ giới nghiêm": 'Cổng vào chỗ nghỉ sẽ đóng trong khoảng 00:00-5:00',
        "Hút thuốc": 'Không cho phép hút thuốc.',
        "Hủy đặt phòng/ Trả trước": 'Các chính sách hủy và thanh toán trước',
        "Không giới hạn độ tuổi": 'Không có yêu cầu về độ tuổi',
        "Nhóm": None,
        "Thẻ được chấp nhận tại chỗ nghỉ này": None,
        "Thẻ được chấp nhận tại khách sạn này": 'Bankcard',
        "Thời gian yên lặng": None,
        "Tiệc tùng": 'Cho phép tổ chức',
        "Trẻ em và giường": 'Trẻ em dưới 5 tuổi',
        "Vật nuôi": 'Cho phép mang vật nuôi',
        "Đặt cọc đề phòng hư hại có thể hoàn lại": 'Yêu cầu VND 1.000.000 tiền đặt cọc đề phòng hư hại khi đến nghỉ.',
        'room_service_included': 'Không cần thanh toán trước - thanh toán tại chỗ nghỉ\nKhông cần thẻ tín dụng'
    }

    # Input thông tin quan tâm về review

}

In [18]:
feature_star = []
for data in results:
    if data['id'] in ids_result:
        feature_star.append(data)

In [19]:
len(feature_star)

1000

In [20]:
# 2. Filter star rating
if user_input.get('stars_rating') not in [None, '', []]:
    result_stars_hotels_score = get_score_price_star.get_score_rating(user_input, feature_star)


In [24]:
with open('D:\\graduate_dissertation\\final\\dataset\\info_hotel\\hotel_data_room.json', 'r', encoding='utf-8') as f:
    feature_detail_room = json.load(f)

In [28]:
detail_room = []
for detail_room_ in feature_detail_room:
    if detail_room_['id'] in ids_result:
        detail_room.append(detail_room_)

In [29]:
detail_room

[{'id': '10000593',
  'name': 'Bungalow Nhìn ra vườn',
  'room_id': 'RD1000059301',
  'cap_room_type': 'Double Room',
  'room_type': 'Bungalow',
  'room_level': 'Standard',
  'original_price': 'VND 1.832.536',
  'capacity': 2.0,
  'room_area(m²)': 32.0,
  'bed_types': {'giường đôi cực lớn': 1},
  'room_view': ['Ban công',
   'Nhìn ra hồ',
   'Nhìn ra vườn',
   'Nhìn ra núi',
   'Hướng nhìn sân trong'],
  'breakfast': True,
  'room_amenities': ['Lối vào riêng',
   'Ổ điện gần giường',
   'Sàn trải thảm',
   'Khu vực ăn uống ngoài trời',
   'Giá phơi quần áo',
   'Tiện nghi BBQ',
   'Sàn lát gỗ',
   'Hoàn toàn nằm ở tầng trệt',
   'Nhìn ra núi',
   'Nhìn ra hồ',
   'Hướng nhìn sân trong',
   'Hút thuốc:  Không hút thuốc',
   'Két an toàn',
   'Hệ thống sưởi',
   'Sản phẩm lau rửa',
   'Màn chống muỗi',
   'Không gây dị ứng',
   'Máy điều hòa độc lập cho từng phòng',
   'Điều hòa không khí',
   '1 bungalow nguyên căn',
   'Quạt máy',
   'Phòng tắm riêng',
   'Bàn làm việc',
   '

In [36]:
# 3. Filter theo giá
price = user_input.get('price')
if price not in [None, '', [], (None, None)]:
    result_room_price_score = get_score_price_star.get_price_score(user_input, detail_room)
    result_hotel_price_score = get_score_price_star.get_max_room_score_per_hotel(result_room_price_score)
else: 
    result_hotel_price_score = []
    for price in detail_room:
        result_hotel_price_score.append({
            'id': price['id'], 
            'hotel_score_price' : 1
        })




In [37]:
result_hotel_price_score

[{'id': '10000593', 'hotel_score_price': 0.9847},
 {'id': '10000937', 'hotel_score_price': 0.25},
 {'id': '10001167', 'hotel_score_price': 0.1042},
 {'id': '10001434', 'hotel_score_price': 0.875},
 {'id': '10001679', 'hotel_score_price': 0.8333},
 {'id': '10003086', 'hotel_score_price': 0.7021},
 {'id': '10003813', 'hotel_score_price': 0.7708},
 {'id': '10004054', 'hotel_score_price': 0.5028},
 {'id': '10006504', 'hotel_score_price': 0.6983},
 {'id': '10006969', 'hotel_score_price': 0.675},
 {'id': '10007210', 'hotel_score_price': 0.8435},
 {'id': '10007353', 'hotel_score_price': 0.9921},
 {'id': '10007578', 'hotel_score_price': 0.8958},
 {'id': '10007720', 'hotel_score_price': 0.8958},
 {'id': '10009281', 'hotel_score_price': 0.6833},
 {'id': '10009298', 'hotel_score_price': 0.9347},
 {'id': '10009426', 'hotel_score_price': 0.9625},
 {'id': '10012226', 'hotel_score_price': 0.6231},
 {'id': '10012325', 'hotel_score_price': 0.9958},
 {'id': '10012516', 'hotel_score_price': 0.6813},
 {'i

In [34]:
result_hotel_price_score = []
for price in detail_room:
    result_hotel_price_score.append({
        'id': price['id'], 
        'hotel_score_price' : 1
    })